In [1]:
import pandas as pd
#!pip install pyreadstat
#!pip install py7zr
import py7zr
import calendar
import os

In [2]:
features_group = pd.read_excel(r'.\codes\feature_groups.xlsx')
demo_order = pd.read_excel(r'.\codes\demo_order.xlsx')
periods = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='period_lbl')
time_period_type = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='time_period_type')
year = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='year')
segments_order = pd.read_excel(r'.\codes\segments_order.xlsx')

In [3]:
example = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'example')
legends = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'legends')

In [4]:
#archive = py7zr.SevenZipFile('../data_in/out-for-datatile.7z', 'r')  
#archive.extractall(path="/tmp")
path = '/tmp/out-for-datatile'
files = os.listdir(path)
files


['br_allshops.parquet',
 'br_tot.parquet',
 'deterg_allshops.parquet',
 'deterg_tot.parquet',
 'diapers_allshops.parquet',
 'diapers_tot.parquet',
 'femcare_allshops.parquet',
 'femcare_tot.parquet',
 'haircond_allshops.parquet',
 'haircond_tot.parquet',
 'shampoo_allshops.parquet',
 'shampoo_tot.parquet']

In [5]:
cat_dict = {'br':'MALE B&R', 'deterg': 'Laundry Detergents','diapers': 'Diapers','femcare': 'Feminine Care', 
                'haircond': 'Hair Conditioners', 'shampoo':'Shampoos' }
time_period_type_dict ={ 24 :'2MATs/ 104 we', 12:'MAT/ 52 we', 3:'3MMT/ 12we' }

In [6]:
def get_period_lbl(time_period_type, year, month):
    return f"{time_period_type.split('/')[0]}: {calendar.month_abbr[month]} {year}"  

In [7]:
def get_df_in_v2(path, file):
    categoty  = cat_dict[file.split('_')[0]]
    type_of_file = file.split('_')[1]
    df_in = pd.read_parquet(f'{path}\\{file}')
    df_in = df_in.rename({'Category Name':'Product Name', 'Buyer Group Name':'buyers_gr_label'
                           }, axis=1 )
    df_in['Category Name'] = [categoty] * len( df_in)
    
    col_to_int = ['duration', 'year', 'month']
    for col in col_to_int:
        df_in[col] = df_in[col].astype(int)
    
    df_in['time_period_type'] = df_in['duration'].map(time_period_type_dict) 
    #Ниже нужно оптимизировать функцию, по 10 секунд выполняется
    df_in['period_lbl'] = df_in[['time_period_type', 'year', 'month']].apply(lambda row:get_period_lbl(*row), axis=1)
    print(sum(df_in['time_period_type'].isnull()))   
    print(df_in.shape)
    df_in = df_in.merge(features_group, on =['Category Name', 'Product Name'], how = 'left') 
    print(df_in.shape)
    df_in = df_in.merge(demo_order, on =['buyers_gr_label'], how = 'left')
    print(df_in.shape)
    df_in = df_in.merge(periods, on =['period_lbl'], how = 'left') 
    print(df_in.shape)
    df_in = df_in.merge(time_period_type, on =['time_period_type'], how = 'left')
    print(df_in.shape)
    df_in = df_in.merge(year, on =['year'], how = 'left')
    print(df_in.shape)
    df_in = df_in.merge(segments_order, on =['Segment'], how = 'left')
    print(df_in.shape)
    return df_in
    

In [8]:
df_in_one = get_df_in_v2(path, files[0])

0
(210575, 34)
(210575, 42)
(210575, 47)
(210575, 49)
(210575, 50)
(210575, 51)
(210575, 52)


In [9]:
df_in_two = get_df_in_v2(path, files[1])

0
(8777, 34)
(8777, 42)
(8777, 47)
(8777, 49)
(8777, 50)
(8777, 51)
(8777, 52)


In [10]:
for col in df_in_one:
    if sum(df_in_one[col].isnull()) >0:
        print(col,sum(df_in_one[col].isnull()))

In [11]:
sorted(df_in_one.columns)

['Buyer Group ID',
 'Category Name',
 'Product Name',
 'Segment',
 'Vendor Product ID',
 'avg_pack_size',
 'avg_price_g',
 'avg_price_pack',
 'avg_price_su',
 'buyers',
 'buyers_gr_label',
 'buyers_raw',
 'category_code',
 'demo_code',
 'demo_hier',
 'demo_lvls',
 'duration',
 'feature',
 'feature_code',
 'frequency',
 'full_label',
 'label_num',
 'month',
 'month_end',
 'penetration',
 'period_code',
 'period_lbl',
 'population',
 'position_name_shop',
 'product_code',
 'product_hier',
 'product_lvls',
 'segment_code',
 'socdem_gr',
 'spend_per_buyer',
 'spend_per_trip',
 'time_period_code',
 'time_period_type',
 'trips',
 'trips_raw',
 'value',
 'volume_SU',
 'volume_gl',
 'volume_pack',
 'volume_per_buyer_g',
 'volume_per_buyer_pack',
 'volume_per_buyer_su',
 'volume_per_trip_g',
 'volume_per_trip_pack',
 'volume_per_trip_su',
 'year',
 'year_code']

In [12]:
legends[legends['Source']=='metrics'].sort_values(by='Colum')



,Colum,Source,mask
0,average_per_su_local_currency,metrics,avg_price_su
1,average_per_su_usd,metrics,NaN
2,buying_households,metrics,buyers
9,item_buying_rate_local_currency,metrics,NaN
10,item_buying_rate_usd,metrics,NaN
11,loyalty_value_based_local_currency,metrics,NaN
12,loyalty_value_based_usd,metrics,NaN
13,loyalty_volume_based,metrics,NaN
14,occasions,metrics,NaN
15,percent_2__time_buyers,metrics,NaN


In [13]:
df_in_one.columns =[col.lower().replace(' ','_')  for col in  df_in_one.columns]

In [14]:
metric_dict = {}
for m, c in legends.loc[legends['mask'].notna(), ['mask','Colum' ]].values:
    metric_dict[m] = c
metric_dict    

{'avg_price_su': 'average_per_su_local_currency',
 'buyers': 'buying_households',
 'segment_code': 'cat_segment',
 "'category_code'": 'category',
 'demo_code': 'demo_groups',
 'feature_code': 'features',
 'penetration': 'percent_household_penetration',
 'product_code': 'products',
 'frequency': 'purchase_frequency',
 'avg_price_pack': 'purchase_size_su',
 'buyers_raw': 'raw_shoppers',
 'value': 'spend_local_currency',
 'volume_pack': 'volume_physical_units',
 'volume_su': 'volume_su'}

In [15]:
df_in_one = df_in_one.rename(metric_dict, axis=1)

In [16]:
sorted(df_in_one.columns)

['average_per_su_local_currency',
 'avg_pack_size',
 'avg_price_g',
 'buyer_group_id',
 'buyers_gr_label',
 'buying_households',
 'cat_segment',
 'category_code',
 'category_name',
 'demo_groups',
 'demo_hier',
 'demo_lvls',
 'duration',
 'feature',
 'features',
 'full_label',
 'label_num',
 'month',
 'month_end',
 'percent_household_penetration',
 'period_code',
 'period_lbl',
 'population',
 'position_name_shop',
 'product_hier',
 'product_lvls',
 'product_name',
 'products',
 'purchase_frequency',
 'purchase_size_su',
 'raw_shoppers',
 'segment',
 'socdem_gr',
 'spend_local_currency',
 'spend_per_buyer',
 'spend_per_trip',
 'time_period_code',
 'time_period_type',
 'trips',
 'trips_raw',
 'vendor_product_id',
 'volume_gl',
 'volume_per_buyer_g',
 'volume_per_buyer_pack',
 'volume_per_buyer_su',
 'volume_per_trip_g',
 'volume_per_trip_pack',
 'volume_per_trip_su',
 'volume_physical_units',
 'volume_su',
 'year',
 'year_code']

In [17]:
legends['is_got'] = legends['Colum'].apply(lambda x: x in df_in_one.columns )

In [18]:
legends[legends['is_got']==False]

,Colum,Source,mask,is_got
1,average_per_su_usd,metrics,NaN,False
4,category,feature_groups,'category_code',False
9,item_buying_rate_local_currency,metrics,NaN,False
10,item_buying_rate_usd,metrics,NaN,False
11,loyalty_value_based_local_currency,metrics,NaN,False
12,loyalty_value_based_usd,metrics,NaN,False
13,loyalty_volume_based,metrics,NaN,False
14,occasions,metrics,NaN,False
15,percent_2__time_buyers,metrics,NaN,False
22,purchase_size_local_currency,metrics,NaN,False


In [19]:
df_out = pd.read_spss('..//PG_cat1_chunk0.sav')

In [20]:
sorted(df_in_one.columns)

['average_per_su_local_currency',
 'avg_pack_size',
 'avg_price_g',
 'buyer_group_id',
 'buyers_gr_label',
 'buying_households',
 'cat_segment',
 'category_code',
 'category_name',
 'demo_groups',
 'demo_hier',
 'demo_lvls',
 'duration',
 'feature',
 'features',
 'full_label',
 'label_num',
 'month',
 'month_end',
 'percent_household_penetration',
 'period_code',
 'period_lbl',
 'population',
 'position_name_shop',
 'product_hier',
 'product_lvls',
 'product_name',
 'products',
 'purchase_frequency',
 'purchase_size_su',
 'raw_shoppers',
 'segment',
 'socdem_gr',
 'spend_local_currency',
 'spend_per_buyer',
 'spend_per_trip',
 'time_period_code',
 'time_period_type',
 'trips',
 'trips_raw',
 'vendor_product_id',
 'volume_gl',
 'volume_per_buyer_g',
 'volume_per_buyer_pack',
 'volume_per_buyer_su',
 'volume_per_trip_g',
 'volume_per_trip_pack',
 'volume_per_trip_su',
 'volume_physical_units',
 'volume_su',
 'year',
 'year_code']